In [38]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score


In [39]:
# Define the dataset class
class EnergyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X.iloc[idx]
        y = self.y.iloc[idx]
        return X.values.reshape(1, -1), y.values  # Adding an extra dimension for sequence length

In [40]:
# Load the data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [41]:
# Create the dataset
train_dataset = EnergyDataset(X_train, y_train)
val_dataset = EnergyDataset(X_val, y_val)
test_dataset = EnergyDataset(X_test, y_test)

In [42]:
# Data Loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [43]:
import torch.nn as nn

# Define the LSTM-QCNN model
class LSTM_QCNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, lstm_layers, fc_dim):
        super(LSTM_QCNN, self).__init__()
        
        # Define LSTM layer(s)
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, 
                            num_layers=lstm_layers, batch_first=True, dropout=0.3)
        
        # Define QCNN layers
        self.fc1 = nn.Linear(hidden_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        # LSTM layer: process sequential data
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # Use the last output of the LSTM
        
        # QCNN fully connected layers
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.fc5(x)
        return x

In [44]:
# Set hyperparameters
learning_rate = 0.001
n_epochs = 200

In [45]:
# Define hyperparameters for the LSTM-QCNN model
input_dim = X_train.shape[1]   # Number of features in input
hidden_dim = 128               # Hidden dimension for the LSTM layer
lstm_layers = 2                # Number of LSTM layers
fc_dim = 256                   # Dimension for fully connected layers in QCNN part

# Initialize model, optimizer, and loss function
model = LSTM_QCNN(input_dim=input_dim, hidden_dim=hidden_dim, lstm_layers=lstm_layers, fc_dim=fc_dim)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()


In [46]:
# Lists to log training and validation loss and accuracy
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

In [47]:
# Train the model
for epoch in range(n_epochs):
    model.train()  
    epoch_loss = 0
    epoch_train_r2 = 0  

    for batch in train_loader:
        X, y = batch
        X = X.clone().detach().float()  
        y = y.clone().detach().float()  

        optimizer.zero_grad()  
        outputs = model(X)  
        loss = loss_fn(outputs, y)
        loss.backward()  
        optimizer.step()  

        # Accumulate the loss for this batch
        epoch_loss += loss.item()

        # Accumulate R² score for training accuracy
        epoch_train_r2 += r2_score(y.numpy(), outputs.detach().numpy())

    # Calculate the average loss and accuracy for this epoch
    average_loss = epoch_loss / len(train_loader)
    train_losses.append(average_loss)  
    average_train_r2 = epoch_train_r2 / len(train_loader)  
    train_accuracy = average_train_r2 * 100  
    train_accuracies.append(train_accuracy)  

    # Validate the model
    model.eval()  
    val_loss = 0  
    epoch_val_r2 = 0  

    with torch.no_grad():
        for batch in val_loader:
            X, y = batch
            X = X.clone().detach().float()
            y = y.clone().detach().float()  

            outputs = model(X)  
            loss = loss_fn(outputs, y)  
            val_loss += loss.item()  

            # Accumulate R² score for validation accuracy
            epoch_val_r2 += r2_score(y.numpy(), outputs.detach().numpy())

        # Calculate the average validation loss and accuracy for this epoch
        average_val_loss = val_loss / len(val_loader)
        val_losses.append(average_val_loss)
        average_val_r2 = epoch_val_r2 / len(val_loader)  
        val_accuracy = average_val_r2 * 100  
        val_accuracies.append(val_accuracy)
    # Print training loss for this epoch
    print(f'Epoch [{epoch + 1}/{n_epochs}], Train Loss: {average_loss}')

Epoch [1/200], Train Loss: 0.029633477690381955
Epoch [2/200], Train Loss: 0.0036036595932118147
Epoch [3/200], Train Loss: 0.002991866680166309
Epoch [4/200], Train Loss: 0.0027298397829906705
Epoch [5/200], Train Loss: 0.0024894981813068213
Epoch [6/200], Train Loss: 0.0023090533423905336
Epoch [7/200], Train Loss: 0.002355882569179461
Epoch [8/200], Train Loss: 0.0022782519776337676
Epoch [9/200], Train Loss: 0.002386213583099769
Epoch [10/200], Train Loss: 0.0022089677181558118
Epoch [11/200], Train Loss: 0.0022174328077776357
Epoch [12/200], Train Loss: 0.0021137453542355656
Epoch [13/200], Train Loss: 0.0019390044819713587
Epoch [14/200], Train Loss: 0.0017485499499724495
Epoch [15/200], Train Loss: 0.0013052824350159537
Epoch [16/200], Train Loss: 0.001234400579769582
Epoch [17/200], Train Loss: 0.0011312478188409518
Epoch [18/200], Train Loss: 0.0009566111974460661
Epoch [19/200], Train Loss: 0.0010621913768244225
Epoch [20/200], Train Loss: 0.0009644203878528739
Epoch [21/200]

In [48]:
# Save trained model and metrics
torch.save({
    'model_state_dict': model.state_dict(),
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies
}, 'models/qcnn_lstm_model.pth')

In [49]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np  

# Test the model on the test set
model.eval()  # Set the model to evaluation mode
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

with torch.no_grad():
    y_test_pred = model(torch.tensor(X_test.values, dtype=torch.float32).unsqueeze(1)).flatten().numpy()
    
    # Calculate MSE, MAE, RMSE, R² score, and MAPE
    test_mse = mean_squared_error(y_test.values, y_test_pred)
    test_r2 = r2_score(y_test.values, y_test_pred)
    test_mae = mean_absolute_error(y_test.values, y_test_pred)
    test_rmse = mean_squared_error(y_test.values, y_test_pred, squared=False)
    test_mape = np.mean(np.abs((y_test.values - y_test_pred) / y_test.values)) * 100

    # Calculate accuracy as R² score in percentage
    test_accuracy = test_r2 * 100
    
    print(f'Mean Squared Error on test data: {test_mse}')
    print(f'R² score (Accuracy) on test data: {test_r2}')
    print(f'Accuracy: {test_accuracy}%')
    print(f'Mean Absolute Error (MAE) on test data: {test_mae}')
    print(f'Root Mean Squared Error (RMSE) on test data: {test_rmse}')
    print(f'Mean Absolute Percentage Error (MAPE) on test data: {test_mape}%')


c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Mean Squared Error on test data: 0.0004425968920501068
R² score (Accuracy) on test data: 0.9887141720202578
Accuracy: 98.87141720202578%
Mean Absolute Error (MAE) on test data: 0.01543800679080574
Root Mean Squared Error (RMSE) on test data: 0.02103798688206899
Mean Absolute Percentage Error (MAPE) on test data: 86.92679167412251%
